In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [2]:
# Loading movie data from CSV files
movies = pd.read_csv('/content/tmdb_5000_credits.csv')
credits = pd.read_csv('/content/tmdb_5000_movies.csv')

In [ ]:
# Display the first 2 rows of the movies DataFrame
movies.head(2)

In [ ]:
# Check the shape of the movies DataFrame
movies.shape

(4803, 20)

In [ ]:

# Display the first few rows of the credits DataFram
credits.head()

In [4]:
# Merge movies and credits DataFrames on the 'title' column
movies = movies.merge(credits,on='title')

In [ ]:
movies.head()
# budget
# homepage
# id
# original_language
# original_title
# popularity
# production_comapny
# production_countries
# release-date(not sure)

In [6]:
# Select relevant columns from the merged DataFrame
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [8]:
#This function evaluates an expression node or a string consisting of a Python literal or container display.
import ast

In [9]:
# Function to convert text data (string representation of list of dictionaries) to list of names
def convert(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name'])
    return L

In [10]:

# Drop rows with missing values
movies.dropna(inplace=True)

In [ ]:
# Apply the convert function to 'genres' column
movies['genres'] = movies['genres'].apply(convert)
movies.head()

In [ ]:
# Apply the convert function to 'keywords' column
movies['keywords'] = movies['keywords'].apply(convert)
movies.head()

In [ ]:
import ast
ast.literal_eval('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

In [14]:
# Function is used to fetch top Three cast
def convert3(text):
    L = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 3:
            L.append(i['name'])
        counter+=1
    return L

In [ ]:
movies['cast'] = movies['cast'].apply(convert)
movies.head()

In [16]:
# Function to extract only the first 3 elements from the lis
movies['cast'] = movies['cast'].apply(lambda x:x[0:3])

In [17]:
# Function is used to fetch director
def fetch_director(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L

In [18]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [ ]:
#movies['overview'] = movies['overview'].apply(lambda x:x.split())
movies.sample(5)

In [20]:
# Function is used to remove spaces between names,genres,keywords
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [21]:
movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)

In [ ]:
movies.head()

In [22]:
# Split 'overview' column into list of words
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [23]:
# Merging few columns and forming a new columns tags
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [24]:
# Dropping the remaning columns
new = movies.drop(columns=['overview','genres','keywords','cast','crew'])
#new.head()

In [ ]:
# converting the tags column to string
new['tags'] = new['tags'].apply(lambda x: " ".join(x))
new.head()

In [26]:
#Importing CountVectorizer from sklearn to covert all character into vector form
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')


In [27]:
# coverting into vector
vector = cv.fit_transform(new['tags']).toarray()

In [ ]:
vector.shape

In [29]:
# importing cosine_similarity to calculate distance between vectors
from sklearn.metrics.pairwise import cosine_similarity

In [30]:
similarity = cosine_similarity(vector)

In [ ]:
similarity

In [32]:
#this line of code is used to find the index of the row in the DataFrame new that corresponds to the movie titled 'The Lego Movie'.
new[new['title'] == 'The Lego Movie'].index[0]

744

In [33]:
# Function to recommend similar movies based on input movie title
def recommend(movie):
    index = new[new['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(new.iloc[i[0]].title)



In [35]:
# Recommending movies which are like batman
recommend('Batman')

Batman
Batman & Robin
The Dark Knight Rises
Batman Begins
Batman Returns


In [39]:
#Pickle is a Python module used for serializing and deserializing Python objects
import pickle

In [40]:
# Save new DataFrame and similarity matrix to pickle files
pickle.dump(new,open('movie_list.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))